## その野球選手がスーパースター（年俸5000万円以上）かどうかの二値分類予測モデル

In [16]:
# 必要なライブラリーの読み込み
import numpy as np
import pandas as pd
# 小数点の桁を第一位までにする
pd.options.display.precision = 1
pd.set_option('display.max_columns', 82)

In [17]:
# 学習データ、テストデータの読み込み
# "-"は欠損値として取り扱う
train = pd.read_csv("bb_train.csv", na_values=["-"])
test = pd.read_csv("bb_test.csv", na_values=["-"])
# floatになってしまっている目的変数をintにする
train["SS"] = train["SS"].fillna(0).astype(int)

In [18]:
print(type(train["SS"][0]))

<class 'numpy.int64'>


In [ ]:
## データ理解(EDA)

In [9]:
# train = train[~train["2020打率"].isna()]
# train["2020打率"].mean()

0.2149067796610171

In [19]:
train.describe()

,SS,年齢,身長,体重,外国人,2020年打席あり,2019年打席あり,2018年打席あり,2020打率,2020試合,2020打席数,2020打数,2020得点,2020安打,2020二塁打,2020三塁打,2020本塁打,2020塁打,2020打点,2020盗塁,2020盗塁刺,2020犠打,2020犠飛,2020四球,2020敬遠,2020死球,2020三振,2020併殺打,2020出塁率,2020長打率,2019打率,2019試合,2019打席数,2019打数,2019得点,2019安打,2019二塁打,2019三塁打,2019本塁打,2019塁打,2019打点,2019盗塁,2019盗塁刺,2019犠打,2019犠飛,2019四球,2019敬遠,2019死球,2019三振,2019併殺打,2019出塁率,2019長打率,2018打率,2018試合,2018打席数,2018打数,2018得点,2018安打,2018二塁打,2018三塁打,2018本塁打,2018塁打,2018打点,2018盗塁,2018盗塁刺,2018犠打,2018犠飛,2018四球,2018敬遠,2018死球,2018三振,2018併殺打,2018出塁率,2018長打率
count,365.0,365.0,365.0,365.0,3.6e+02,365.0,365.0,365.0,2.4e+02,236.0,236.0,236.0,236.0,236.0,236.0,236.0,236.0,236.0,236.0,236.0,236.0,236.0,236.0,236.0,236.0,236.0,236.0,236.0,2.4e+02,236.0,211.0,211.0,211.0,211.0,211.0,211.0,211.0,211.0,211.0,211.0,211.0,211.0,211.0,211.0,211.0,211.0,211.0,211.0,211.0,211.0,211.0,211.0,181.0,181.0,181.0,181.0,181.0,181.0,181.0,181.0,181.0,181.0,181.0,181.0,181.0,181.0,181.0,181.0,181.0,181.0,181.0,181.0,181.0,181.0
mean,0.2,25.8,179.4,84.1,8.2e-02,0.6,0.6,0.5,2.1e-01,54.5,160.4,141.0,18.0,35.8,6.4,0.7,4.1,55.9,17.7,2.6,1.1,2.2,0.9,14.8,0.7,1.4,31.2,3.0,2.8e-01,0.3,0.2,67.5,215.2,189.9,24.5,49.0,8.5,0.9,5.6,76.0,23.4,3.3,1.5,3.2,1.2,18.8,1.1,2.1,40.7,4.0,0.3,0.3,0.2,72.2,238.4,209.6,27.6,55.2,9.6,1.3,6.3,86.3,26.9,3.7,1.6,3.6,1.5,21.3,1.1,2.4,42.5,4.2,0.3,0.4
std,0.4,5.2,5.7,9.4,2.8e-01,0.5,0.5,0.5,7.9e-02,36.9,158.9,138.9,19.9,39.7,7.8,1.3,6.8,65.9,23.3,5.4,1.7,3.5,1.4,18.9,1.9,2.0,30.2,4.0,9.2e-02,0.1,0.1,47.9,207.9,182.1,27.0,51.7,9.8,1.4,9.0,84.4,28.5,6.5,2.7,4.7,1.8,23.3,2.0,2.8,38.2,4.8,0.1,0.3,0.1,46.6,208.5,181.2,28.7,52.8,9.9,2.0,9.7,88.3,30.4,6.5,2.6,5.9,2.0,24.8,1.8,3.4,36.4,4.5,0.1,0.3
min,0.0,16.0,164.0,65.0,0.0e+00,0.0,0.0,0.0,0.0e+00,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0e+00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,22.0,175.0,78.0,0.0e+00,0.0,0.0,0.0,1.8e-01,22.8,32.8,30.5,3.0,5.0,1.0,0.0,0.0,6.0,1.8,0.0,0.0,0.0,0.0,2.0,0.0,0.0,7.0,0.0,2.4e-01,0.2,0.2,24.0,36.5,32.5,4.0,6.0,1.0,0.0,0.0,8.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,8.5,0.5,0.2,0.3,0.2,28.0,47.0,41.0,5.0,9.0,1.0,0.0,0.0,11.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,12.0,1.0,0.2,0.3
50%,0.0,25.0,179.0,83.0,0.0e+00,1.0,1.0,0.0,2.3e-01,49.5,92.0,80.0,10.0,17.0,3.0,0.0,1.0,25.0,7.0,0.0,0.0,1.0,0.0,6.5,0.0,1.0,21.0,2.0,2.9e-01,0.3,0.2,60.0,136.0,124.0,14.0,30.0,5.0,0.0,2.0,43.0,11.0,1.0,0.0,1.0,0.0,9.0,0.0,1.0,29.0,2.0,0.3,0.3,0.2,72.0,166.0,145.0,17.0,35.0,6.0,0.0,2.0,54.0,16.0,1.0,1.0,1.0,1.0,12.0,0.0,1.0,31.0,3.0,0.3,0.3
75%,0.0,29.0,183.0,90.0,0.0e+00,1.0,1.0,1.0,2.7e-01,86.0,270.5,234.2,28.2,58.2,10.0,1.0,5.0,84.5,23.0,3.0,2.0,3.0,1.0,23.0,1.0,2.0,49.2,4.0,3.4e-01,0.4,0.3,110.5,373.0,325.0,39.0,87.0,13.0,1.0,6.0,125.5,33.0,3.5,2.0,4.0,2.0,27.0,1.0,3.0,64.5,6.0,0.3,0.4,0.3,114.0,421.0,379.0,46.0,99.0,15.0,2.0,8.0,155.0,45.0,4.0,2.0,4.0,2.0,32.0,2.0,3.0,68.0,6.0,0.3,0.4
max,1.0,43.0,201.0,119.0,1.0e+00,1.0,1.0,1.0,4.3e-01,120.0,546.0,478.0,90.0,146.0,31.0,8.0,32.0,266.0,108.0,50.0,9.0,16.0,10.0,91.0,17.0,13.0,150.0,21.0,4.7e-01,1.0,1.0,143.0,648.0,586.0,112.0,168.0,43.0,7.0,43.0,291.0,123.0,41.0,16.0,28.0,10.0,110.0,12.0,16.0,184.0,22.0,1.0,4.0,1.0,143.0,675.0,578.0,130.0,178.0,37.0,14.0,47.0,319.0,131.0,33.0,13.0,30.0,13.0,130.0,10.0,19.0,138.0,24.0,1.0,4.0


In [20]:
# 学習データを特徴量と目的変数に分ける
train_x = train.drop("SS", axis=1)
train_y = train["SS"]

In [21]:
# テストデータから目的変数を除く
test_x = test.drop("SS", axis=1)

In [ ]:
# from sklearn.model_selection import KFold

In [22]:
# 予測と関係がないと思われる特徴量を除外する
train_x.columns
train_x = train_x.drop(['選手名','生年月日','血液型','出身地'], axis=1)
test_x = test_x.drop(['選手名','生年月日','血液型','出身地'], axis=1)

In [23]:
train_x.columns

Index(['現チーム', '守備', '年齢', '身長', '体重', '投打', '外国人', '地域', '2020年打席あり',
       '2019年打席あり', '2018年打席あり', '2020打率', '2020試合', '2020打席数', '2020打数',
       '2020得点', '2020安打', '2020二塁打', '2020三塁打', '2020本塁打', '2020塁打', '2020打点',
       '2020盗塁', '2020盗塁刺', '2020犠打', '2020犠飛', '2020四球', '2020敬遠', '2020死球',
       '2020三振', '2020併殺打', '2020出塁率', '2020長打率', '2019打率', '2019試合',
       '2019打席数', '2019打数', '2019得点', '2019安打', '2019二塁打', '2019三塁打',
       '2019本塁打', '2019塁打', '2019打点', '2019盗塁', '2019盗塁刺', '2019犠打', '2019犠飛',
       '2019四球', '2019敬遠', '2019死球', '2019三振', '2019併殺打', '2019出塁率', '2019長打率',
       '2018打率', '2018試合', '2018打席数', '2018打数', '2018得点', '2018安打', '2018二塁打',
       '2018三塁打', '2018本塁打', '2018塁打', '2018打点', '2018盗塁', '2018盗塁刺', '2018犠打',
       '2018犠飛', '2018四球', '2018敬遠', '2018死球', '2018三振', '2018併殺打', '2018出塁率',
       '2018長打率'],
      dtype='object')

In [24]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# それぞれの糧切り変数にlabel encodingを適用する
for c in ['現チーム', '守備', '投打', '地域']:
    # 学習データに基づいてどう変換するかを定める
    le = LabelEncoder()
#     le.fit(train_x[c].fillna("NA"))
    le.fit(train_x[c]("NA"))

    
    # 学習データ、テストデータを変換する
#     train_x[c] = le.transform(train_x[c].fillna("NA"))
    train_x[c] = le.transform(train_x[c])

#     test_x[c] = le.transform(test_x[c].fillna("NA"))
    test_x[c] = le.transform(test_x[c])
    

In [ ]:

# %conda install -c conda-forge xgboost

In [ ]:
from xgboost import XGBClassifier

In [ ]:
# モデルの作成および学習データを与えての学習
model = XGBClassifier(n_estimators=20, random_state=70,max_depth=3, min_child_weight=3)
model.fit(train_x, train_y)

In [ ]:
pred = model.predict_proba(test_x)[:, 1]

In [ ]:
pred

In [ ]:
pred_label = np.where(pred > 0.5, 1, 0)

In [ ]:
pred_label

In [ ]:
submission = pd.DataFrame({"選手名": test["選手名"], "SS": pred_label})

In [ ]:
submission.to_csv("submission_first.csv", index=False)

In [ ]:
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import KFold

In [ ]:
#　各foldのスコアを保存するリスト
scores_accuracy = []
scores_logloss = []

In [ ]:
# クロスバリデーションを行う
# 学習データを4つに分割し、うち一つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx in kf.split(train_x):
    # 学習データを学習データとバリデーションデータに分ける
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    
    # モデルの学習を行う
    model = XGBClassifier(n_estimators=20, random_state=71)
    model.fit(tr_x, tr_y)
    
    # バリデーションデータの予測値を確率で出力する
    va_pred = model.predict_proba(va_x)[:, 1]
    #バリデーションデータでのスコアを計算する
    logloss = log_loss(va_y, va_pred)
    accuracy = accuracy_score(va_y, va_pred > 0.5)
    
    # そのfoldのスコアを保存する
    scores_logloss.append(logloss)
    scores_accuracy.append(accuracy)

In [ ]:
logloss = np.mean(scores_logloss)
accuracy = np.mean(scores_accuracy)
print(f"logloss: {logloss:.4f}, accuracy: {accuracy:.4f}")
# logloss: 0.1942, accuracy: 0.9289 max_depth=5, min_child_weight=4.0

In [ ]:
import itertools

In [ ]:
#チューニング候補とするパラメーターを準備する
param_space = {
    "max_depth": [3, 4, 5, 6, 7],
    "min_child_weight": [1.0, 2.0, 3.0, 4.0, 5.0, 6.0]
}

In [ ]:
# 探索するハイパーパラメータの組み合わせ
param_combinations = itertools.product(param_space["max_depth"], param_space["min_child_weight"])

In [ ]:
# 各パラメーターの組み合わせ、それに対するスコアを保存するリスト
params = []
scores = []

In [ ]:
# 各パラメーターの組み合わせごとに、クロスバリデーションで評価を行う
for max_depth, min_child_weight in param_combinations:
    score_folds = []
    # クロスバリデーションを行う
#     学習でエータを4つに分割し、うち１つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す
    kf = KFold(n_splits=4, shuffle=True, random_state=123456)
    for tr_idx, va_idx in kf.split(train_x):
        # 学習データを学習データとバリデーションデータに分ける
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    
        # モデルの学習を行う
        model = XGBClassifier(n_estimators=20, random_state=71,
                             max_depth=max_depth, min_child_weight=min_child_weight)
        model.fit(tr_x, tr_y)
    
        # バリデーションデータの予測値を確率で出力する
        va_pred = model.predict_proba(va_x)[:, 1]
        #バリデーションデータでのスコアを計算する
        logloss = log_loss(va_y, va_pred)
        score_folds.append(logloss)
    
    # 各foldのスコアを平均する
    score_mean = np.mean(score_folds)
    
    #パラメーターの組み合わせ、それに対するスコアを保存する
    params.append((max_depth, min_child_weight))
    scores.append(score_mean)
#     accuracy = accuracy_score(va_y, va_pred > 0.5)
    
# そのfoldのスコアを保存する
best_idx = np.argsort(scores)[0]
best_param = params[best_idx]
print(f"max_depth: {best_param[0]}, min_child_weight: {best_param[1]}")
#         scores_logloss.append(logloss)
#         scores_accuracy.append(accuracy)
print(params)
print(scores)